In [137]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score

In [270]:
df=pd.read_csv("D:/anaconda/udemy/IBM_machine_learning/data.csv")
# df.head()
df=df.drop(columns=['date'])
sorted(df.yr_built.unique())
df.yr_renovated.unique()
df.describe()
# df=df.drop(columns=['street','city','statezip','country'])
# df.head()
# # df.info()
# df.dropna(axis=0)
df.country.unique()

array(['USA'], dtype=object)

In [30]:
def yrbuilt(yr):
    if yr<1960:
        return 0
    if 1960<=yr<2000:
        return 1
    else:
        return 2
def yrrenv(yr):
    if yr==0:
        return 0
    else:
        return 1

df.yr_built=df.yr_built.map(yrbuilt)
df.yr_renovated=df.yr_renovated.map(yrrenv)
df


,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated
0,3.130000e+05,3.0,1.50,1340,7912,1.5,0,0,3,1340,0,0,1
1,2.384000e+06,5.0,2.50,3650,9050,2.0,0,4,5,3370,280,0,0
2,3.420000e+05,3.0,2.00,1930,11947,1.0,0,0,4,1930,0,1,0
3,4.200000e+05,3.0,2.25,2000,8030,1.0,0,0,4,1000,1000,1,0
4,5.500000e+05,4.0,2.50,1940,10500,1.0,0,0,4,1140,800,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,3.081667e+05,3.0,1.75,1510,6360,1.0,0,0,4,1510,0,0,1
4596,5.343333e+05,3.0,2.50,1460,7573,2.0,0,0,3,1460,0,1,1
4597,4.169042e+05,3.0,2.50,3010,7014,2.0,0,0,3,3010,0,2,0
4598,2.034000e+05,4.0,2.00,2090,6630,1.0,0,0,3,1070,1020,1,0


In [31]:
df.rename(columns={'yr_built':'age','yr_renovated':'renovated or not'},inplace=True)

In [32]:
df.price=df.price.astype('int')
df

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,age,renovated or not
0,313000,3.0,1.50,1340,7912,1.5,0,0,3,1340,0,0,1
1,2384000,5.0,2.50,3650,9050,2.0,0,4,5,3370,280,0,0
2,342000,3.0,2.00,1930,11947,1.0,0,0,4,1930,0,1,0
3,420000,3.0,2.25,2000,8030,1.0,0,0,4,1000,1000,1,0
4,550000,4.0,2.50,1940,10500,1.0,0,0,4,1140,800,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,308166,3.0,1.75,1510,6360,1.0,0,0,4,1510,0,0,1
4596,534333,3.0,2.50,1460,7573,2.0,0,0,3,1460,0,1,1
4597,416904,3.0,2.50,3010,7014,2.0,0,0,3,3010,0,2,0
4598,203400,4.0,2.00,2090,6630,1.0,0,0,3,1070,1020,1,0


In [33]:

# px.box(df.price)
df_hist=df.drop(columns=['waterfront','view','age','renovated or not'])
df_hist
fig=make_subplots(rows=3,cols=3,subplot_titles=df_hist.columns)
for i in range(1,len(df_hist.columns)+1):
    if i%3!=0:
        fig.add_trace(go.Histogram(x=df_hist[df_hist.columns[i-1]],name=df_hist.columns[i-1]),row=i//3+1,col=i%3)
    if i%3==0:
        fig.add_trace(go.Histogram(x=df_hist[df_hist.columns[i-1]],name=df_hist.columns[i-1]),row=i//3,col=3)
fig


In [262]:
ss=StandardScaler()
poly=PolynomialFeatures()
dummy=df[['waterfront','view','condition','age','renovated or not']]
df_need_ss=df.drop(columns=['waterfront','view','condition','age','renovated or not'])
df_need_ss_after=ss.fit_transform(df_need_ss)
df_need_ss_after=pd.DataFrame(df_need_ss_after,columns=df_need_ss.columns)
df_ss=df_need_ss_after.join(dummy)
df_ss

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,sqft_basement,waterfront,view,condition,age,renovated or not
0,-0.423864,-0.441122,-0.843204,-0.829971,-0.193434,-0.022416,-0.565224,-0.672464,0,0,3,0,1
1,3.249598,1.759705,0.432802,1.568528,-0.161718,0.906555,1.789559,-0.069128,0,4,5,0,0
2,-0.372424,-0.441122,-0.205201,-0.217367,-0.080978,-0.951388,0.119171,-0.672464,0,0,4,1,0
3,-0.234071,-0.441122,0.113800,-0.144686,-0.190145,-0.951388,-0.959621,1.482306,0,0,4,1,0
4,-0.003482,0.659291,0.432802,-0.206984,-0.121306,-0.951388,-0.797222,1.051352,0,0,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,-0.432438,-0.441122,-0.524202,-0.653458,-0.236689,-0.951388,-0.368025,-0.672464,0,0,4,0,1
4596,-0.031271,-0.441122,0.432802,-0.705374,-0.202882,0.906555,-0.426025,-0.672464,0,0,3,1,1
4597,-0.239563,-0.441122,0.432802,0.904009,-0.218462,0.906555,1.371962,-0.672464,0,0,3,2,0
4598,-0.618268,0.659291,-0.205201,-0.051238,-0.229164,-0.951388,-0.878421,1.525401,0,0,3,1,0


In [265]:
rf=pd.DataFrame(columns=['linear','lasso','ridge'],index=['no polynomial','polynomial'])
y=df_ss['price'].to_frame()
x=df_ss.drop(columns='price')
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=4)
clf=GridSearchCV(estimator=LinearRegression(),param_grid={})
clf.fit(x_train,y_train)
rf.loc['no polynomial','linear']=clf.score(x_test,y_test)
cf=pd.DataFrame(columns=df_ss.drop(columns='price').columns,index=['linear','lasso','ridge'])
cf.loc['linear',:]=clf.best_estimator_.coef_
cf

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,sqft_basement,waterfront,view,condition,age,renovated or not
linear,-0.101412,0.076184,0.22182,-0.0445948,0.0307147,0.201151,0.0866821,0.789535,0.0504421,0.0758761,-0.147236,0.0306018
lasso,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ridge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [266]:
clf=GridSearchCV(estimator=Lasso(),
                param_grid={'alpha':[0.01,0.1,1,10],'max_iter':[1000,3000,5000]})
clf.fit(x_train,y_train)
rf.loc['no polynomial','lasso']=clf.score(x_test,y_test)
rf
cf.loc['lasso',:]=clf.best_estimator_.coef_
cf

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,sqft_basement,waterfront,view,condition,age,renovated or not
linear,-0.101412,0.076184,0.22182,-0.0445948,0.0307147,0.201151,0.0866821,0.789535,0.0504421,0.0758761,-0.147236,0.0306018
lasso,-0.0738965,0.0550117,0.393594,-0.0332996,0.0163328,0.0324548,0,0,0.0719932,0.0472137,-0.121315,0
ridge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [267]:
clf=GridSearchCV(estimator=Ridge(),
                param_grid={'alpha':[0.01,0.1,1,10],'max_iter':[1000,3000,5000]})
clf.fit(x_train,y_train)
rf.loc['no polynomial','ridge']=clf.score(x_test,y_test)
cf.loc['ridge',:]=clf.best_estimator_.coef_
cf

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,sqft_basement,waterfront,view,condition,age,renovated or not
linear,-0.101412,0.076184,0.22182,-0.0445948,0.0307147,0.201151,0.0866821,0.789535,0.0504421,0.0758761,-0.147236,0.0306018
lasso,-0.0738965,0.0550117,0.393594,-0.0332996,0.0163328,0.0324548,0,0,0.0719932,0.0472137,-0.121315,0
ridge,-0.101265,0.0765152,0.221111,-0.0445951,0.0309041,0.20095,0.0855828,0.510821,0.0609182,0.0756974,-0.146726,0.0304267


In [254]:
df_poly=df_need_ss.drop(columns='price')
df_poly=poly.fit_transform(df_poly)
df_poly=pd.DataFrame(df_poly)
df_poly_ss=pd.DataFrame(ss.fit_transform(df_poly))
df_poly_ss=df_poly_ss.join(dummy)
x=df_poly_ss
y=df_need_ss.price.to_frame()
y=pd.DataFrame(ss.fit_transform(y))
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=1)
lrp=LinearRegression()
lrp.fit(x_train,y_train)
lrp.score(x_test,y_test)
rf.loc['polynomial','linear']=lrp.score(x_test,y_test)
rf

,linear,lasso,ridge
no polynomial,0.526022,0.520543,0.528389
polynomial,0.434772,NaN,NaN


In [257]:
clf=GridSearchCV(estimator=Lasso(),
                param_grid={'alpha':[0.01,0.1,1,10],'max_iter':[1000,3000,5000]})
clf.fit(x_train,y_train)
rf.loc['polynomial','lasso']=clf.score(x_test,y_test)


,linear,lasso,ridge
no polynomial,0.526022,0.520543,0.528389
polynomial,0.434772,0.470209,NaN


In [259]:
clf=GridSearchCV(estimator=Ridge(),
                param_grid={'alpha':[0.01,0.1,1,10],'max_iter':[1000,3000,5000]})
clf.fit(x_train,y_train)
rf.loc['polynomial','ridge']=clf.score(x_test,y_test)
rf

,linear,lasso,ridge
no polynomial,0.526022,0.520543,0.528389
polynomial,0.434772,0.470209,0.46692
